In [1]:
import os
import matplotlib
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from tqdm.auto import tqdm
import pickle
from sklearn.neighbors import NearestNeighbors
import time
import nlp_utils # our utils

/home/dcor/roeyron/miniconda3/envs/patchscopes/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
results_dir = '/home/dcor/roeyron/TCIE/results/celeba_conditioned_embeddings'

results_fnames = sorted(os.listdir(results_dir), key=lambda name: int(name.split('_')[1].split('.')[0]))
results_fpaths = [os.path.join(results_dir, fname) for fname in results_fnames]

fpath = results_fpaths[4]
# for fpath in tqdm(results_fpaths):
with open(fpath, 'rb') as f:
    df = pickle.load(f)

question = df.question.unique()[0]
question

In [ ]:
###########################
i_layer = -10  # <= 32
i_token = -1

###########################
k = 10
query_ind = 10
###########################



ddl = defaultdict(list)
query_image_inds = np.random.RandomState(6).permutation(len(df))[:20]

for fpath in tqdm(results_fpaths):
    try:
        with open(fpath, 'rb') as f:
            df = pickle.load(f)
    except:
        continue
    question = df.question.unique()[0]
    for query_ind in query_image_inds:
        X = np.array([hs[i_layer][i_token] for hs in df['hidden_states']])
    
        neighbors = NearestNeighbors(n_neighbors=k + 1, metric='cosine')
        neighbors.fit(X)
        query = X[query_ind]
        distances, indices = neighbors.kneighbors([query])
        distances, indices = distances[0, :], indices[0, :]

        ddl['query_ind'].append(query_ind)
        ddl['result_inds'].append(indices)
        ddl['question'].append(question)
        
        # print(f'{query_ind} -------', question)
        # images = [Image.open(df['image_path'].iloc[nn_ind]) for nn_ind in indices]
        # display(Image.fromarray(np.concatenate([img.resize((256, 256)) for img in images], axis=1)))
        
df_search = pd.DataFrame(ddl)
df_search.head()

In [ ]:
df_search.question.unique()

In [ ]:
good_questions_sub_strings = ['expression', 'shirt', 'describe the hair']

def cechk_if_sub_str_in(question, sub_strings):
    for ss in sub_strings:
        if ss in question:
            return True
    return False



for query_ind in df_search.query_ind.unique():
    df_query = df_search[df_search.query_ind == query_ind]
    for _, row in df_query.iterrows():
        # if not cechk_if_sub_str_in(row.question, good_questions_sub_strings):
        #     continue
        print(f' ######### Prompt: {row.question} [{query_ind}]')
        images = [Image.open(df['image_path'].iloc[nn_ind]) for nn_ind in row.result_inds]
        display(Image.fromarray(np.concatenate([img.resize((180, 180)) for img in images], axis=1)))
    

In [ ]:
for _, row in df_query.iterrows():
    print(row['question'])

In [ ]:
df_search.query_ind.unique()

In [ ]:
good_questions_sub_strings